In [1]:
from osgeo import gdal
gdal.UseExceptions()
from matplotlib import pyplot as plt
import boto3
import os
from get_S3_resource import get_s3_resource
from split_image import split_image


In [22]:
import sys
AWSConfig_path = os.path.abspath('AWSConfig')
print(AWSConfig_path)

print(os.path.curdir)
aws_credentials_path = os.path.join(AWSConfig_path, 'credentials')
print("aws_credentials_path", aws_credentials_path)
print(sys.path)

# session, s3_client = get_s3_resource()

/workspaces/DigitalPathology2021/SupervisedLearning/AWSConfig
.
aws_credentials_path /workspaces/DigitalPathology2021/SupervisedLearning/AWSConfig/credentials
['/workspaces/DigitalPathology2021/SupervisedLearning', '/home/vscode/.vscode-server/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles', '/home/vscode/.vscode-server/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles/lib/python', '/opt/conda/envs/pytorch/lib/python39.zip', '/opt/conda/envs/pytorch/lib/python3.9', '/opt/conda/envs/pytorch/lib/python3.9/lib-dynload', '', '/opt/conda/envs/pytorch/lib/python3.9/site-packages', '/opt/conda/envs/pytorch/lib/python3.9/site-packages/IPython/extensions', '/home/vscode/.ipython']


In [3]:
#Then use the session to get the resource
s3 = session.resource('s3')
bucket = s3.Bucket('digpath-data')
paths = []

for idx, x in enumerate(bucket.objects.all()):
    if 'Standard' in x.key:
        paths.append((x.key, x.size*(9.3132*(10**-10))))

paths.sort(key=lambda x: x[1])


prefix = '/vsis3/digpath-data/'

In [4]:
# TODO: create a text file, copy and paste folder name in there and then put if statement if name is not in text file, then go ahead and preprocess
# pickle database to read array directly and save it directly - for massive data let it go
# text file is easier

for path, size in paths:
    if(size > 2):
        break
    
    ds = gdal.Open(prefix + path)
    fn = os.path.splitext(os.path.basename(path))[0]
    
    if not ds:
        print(f"gdal failed to open {prefix}{path}")
        continue

    print(f"Size for {fn} is {ds.RasterXSize} x {ds.RasterYSize} x {ds.RasterCount}")
    split_image(ds, fn)
    # closing the dataset, is implicit by the dataset going out of scope
    ds = None


Size for 84278T_001 is 50375 x 32045 x 3
84278T_001/84278T_001_1024_14336.png uploaded successfull
84278T_001/84278T_001_1024_18432.png uploaded successfull


KeyboardInterrupt: 

In [16]:
import pandas as pd
import boto3

def download_file_from_s3(image_folder, bucket):
    images = []
    local_imgs_dir = f'local_images/{image_folder}'

    s3 = boto3.client('s3')

    if not os.path.exists(local_imgs_dir):
        os.makedirs(local_imgs_dir)

    for x in bucket.objects.all():
        if f'Preprocessing/{image_folder}' in x.key:
            images.append(x.key)

    images.sort(key=lambda x: x[1])

    for image in images:
        s3.download_file('digpath-data', image, f'{local_imgs_dir}{image}')

        # print(image)

        # df = pd.read_csv(image, header=None)
        # df = pd.read_csv(image)
        # ds_array = df.to_numpy()
        # ds_array = image.to_numpy()
        # x_size = ds_array.shape[0]
        # ds_array = ds_array.reshape((x_size, x_size, 3))
        # plt.imshow(ds_array, vmin=0, vmax=255, cmap="gray")
        # plt.show()
        # save_path = os.path.join(f'local_images', image_folder, f'{image}.png')
        # plt.imsave(save_path, ds_array)
    

In [17]:
# from s3_download import download_file_from_s3

download_file_from_s3("84278T_001", bucket)

NoCredentialsError: Unable to locate credentials